First updated @ Nov 2, 2021

Last updated @ Nov 2, 2021

Data Source: https://www.kaggle.com/rounakbanik/the-movies-dataset?select=credits.csv

I downloaded the data from Kaggle directly and now we need to clean up this mess.

In [26]:
import pandas as pd
import json
import ast
from collections import defaultdict

In [3]:
# load data, we only care about the cast and crew for now
df = pd.read_csv('./TheMoviesData/credits.csv')
df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [5]:
# check if there are any empty values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


The info in cast and crew is very disorganized. We only want the gender information, so we need to loop through every cast and crew and count their gender. 

Stringified JSON object to list: https://stackoverflow.com/questions/42631097/convert-stringified-list-of-dictionaries-back-to-a-list-of-dictionaries

In [34]:
df_new = pd.DataFrame(columns=[
    'id', 
    'cast_num', 
    'cast_female_num', 
    'cast_male_num', 
    'crew_num', 
    'crew_female_num', 
    'crew_male_num'
    ])
df_new['id'] = df['id']
df_new.head()

,id,cast_num,cast_female_num,cast_male_num,crew_num,crew_female_num,crew_male_num
0,862,NaN,NaN,NaN,NaN,NaN,NaN
1,8844,NaN,NaN,NaN,NaN,NaN,NaN
2,15602,NaN,NaN,NaN,NaN,NaN,NaN
3,31357,NaN,NaN,NaN,NaN,NaN,NaN
4,11862,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
for index, row in df[:2].iterrows():
    cast_info, crew_info, crew_department = defaultdict(int), defaultdict(int), defaultdict(int)
    cast_list = ast.literal_eval(row['cast'])
    crew_list = ast.literal_eval(row['crew'])

    # count everything in cast
    for cast in cast_list:
        # print(cast)
        if cast['gender'] == 2:
            cast_info['male'] += 1
        elif cast['gender'] == 1:
            cast_info['female'] += 1
        elif cast['gender'] == 0:
            cast_info['unknown'] += 1
        else:
            print(cast)
    df_new.at[index, 'cast_num'] = len(cast_list)
    df_new.at[index, 'cast_female_num'] = cast_info['female']
    df_new.at[index, 'cast_male_num'] = cast_info['male']

    # count evetything in crew
    crew_info['total'] = len(crew_list)
    for crew in crew_list:
        print(crew)
        if crew['gender'] == 2:
            crew_info['male'] += 1
            crew_department[str(crew['department'])+'_male'] += 1
            crew_department[str(crew['department'])] += 1
        elif crew['gender'] == 1:
            crew_info['female'] += 1
            crew_department[str(crew['department'])+'_female'] += 1
            crew_department[str(crew['department'])] += 1
        elif crew['gender'] == 0:
            crew_info['unknown'] += 1
            crew_department[str(crew['department'])+'_unknown'] += 1
            crew_department[str(crew['department'])] += 1
        else:
            print(crew)

    print(crew_department)
    # df_new.at[index, 'crew_num'] = len(crew_list)
    # df_new.at[index, 'crew_female_num'] = crew_info['female']
    # df_new.at[index, 'crew_male_num'] = crew_info['male']

df_new.head()

{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}
{'credit_id': '52fe4284c3a36847f8024f4f', 'department': 'Writing', 'gender': 2, 'id': 12891, 'job': 'Screenplay', 'name': 'Joss Whedon', 'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'}
{'credit_id': '52fe4284c3a36847f8024f55', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'name': 'Andrew Stanton', 'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'}
{'credit_id': '52fe4284c3a36847f8024f5b', 'department': 'Writing', 'gender': 2, 'id': 12892, 'job': 'Screenplay', 'name': 'Joel Cohen', 'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'}
{'credit_id': '52fe4284c3a36847f8024f61', 'department': 'Writing', 'gender': 0, 'id': 12893, 'job': 'Screenplay', 'name': 'Alec Sokolow', 'profile_path': '/v79vlRYi94BZUQnkkyznbGUZLjT.jpg'}
{'credit_id': '52fe4284c3a36847f8024f67', 'department':

,id,cast_num,cast_female_num,cast_male_num,crew_num,crew_female_num,crew_male_num
0,862,13,3,9,106,4,29
1,8844,26,6,9,16,0,14
2,15602,NaN,NaN,NaN,NaN,NaN,NaN
3,31357,NaN,NaN,NaN,NaN,NaN,NaN
4,11862,NaN,NaN,NaN,NaN,NaN,NaN
